In [11]:
import gc
import os

import pandas as pd

from glob import glob

pd.set_option('display.max_rows', 500)

In [7]:
sorted(glob(f"../data/interim/features/train_2017-11-08_0*_channel_*_features.csv"))

['../data/interim/features/train_2017-11-08_0000_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0100_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0200_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0300_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0400_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0500_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0600_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0700_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0800_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0900_channel_1hour_features.csv']

In [8]:
sorted(glob(f"../data/interim/train_2017-11-08_0*00.csv"))

['../data/interim/train_2017-11-08_0000.csv',
 '../data/interim/train_2017-11-08_0100.csv',
 '../data/interim/train_2017-11-08_0200.csv',
 '../data/interim/train_2017-11-08_0300.csv',
 '../data/interim/train_2017-11-08_0400.csv',
 '../data/interim/train_2017-11-08_0500.csv',
 '../data/interim/train_2017-11-08_0600.csv',
 '../data/interim/train_2017-11-08_0700.csv',
 '../data/interim/train_2017-11-08_0800.csv',
 '../data/interim/train_2017-11-08_0900.csv']

In [14]:
dtypes = {
    'ip': 'uint32',
    'app': 'uint16',
    'device': 'uint16',
    'os': 'uint16',
    'channel': 'uint16',
    'is_attributed': 'uint8'
}
to_read = ['app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
to_parse = ['click_time']

In [15]:
train_files =  ['../data/interim/train_2017-11-08_0000.csv',
 '../data/interim/train_2017-11-08_0100.csv',
 '../data/interim/train_2017-11-08_0200.csv',
 '../data/interim/train_2017-11-08_0300.csv',
 '../data/interim/train_2017-11-08_0400.csv']

feature_files = ['../data/interim/features/train_2017-11-08_0000_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0100_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0200_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0300_channel_1hour_features.csv',
 '../data/interim/features/train_2017-11-08_0400_channel_1hour_features.csv']

In [10]:
df_train = pd.read_csv(train_files[0], usecols=to_read, dtype=dtypes, parse_dates=to_parse)
df_feature = pd.read_csv(feature_files[0])
df_feature = df_feature.set_index('channel')
df_train = df_train.join(df_feature, on='channel')
to_drop = ['click_time']
for c in df_train.columns:
    if c.startswith(to_drop_sw):
        to_drop.append(c)

df_train.drop(columns=to_drop, inplace=True)


In [16]:
to_select_sw = tuple([
    'COUNT(clicks)_',
    'PERCENT_TRUE(clicks.is_attributed)_',
    'NUM_UNIQUE(clicks.os)_',
    'NUM_UNIQUE(clicks.app)_',
    'NUM_UNIQUE(clicks.device)_',
    'STD(clicks.device.NUM_UNIQUE(clicks.os))_',
    'STD(clicks.os.NUM_UNIQUE(clicks.device))_',
    'SKEW(clicks.os.COUNT(clicks))_',
    'STD(clicks.app.PERCENT_TRUE(clicks.is_attribut',
    'STD(clicks.device.PERCENT_TRUE(clicks.is_attri',
    'SKEW(clicks.device.COUNT(clicks))_',
    'SKEW(clicks.app.PERCENT_TRUE(clicks.is_attribu'
])

In [6]:
to_drop_sw = tuple([
    'NUM_UNIQUE(clicks.DAY(click_time))',
    'NUM_UNIQUE(clicks.YEAR(click_time))',
    'NUM_UNIQUE(clicks.MONTH(click_time))',
    'NUM_UNIQUE(clicks.WEEKDAY(click_time))',
    'MODE(clicks.DAY(click_time))',
    'MODE(clicks.YEAR(click_time))',
    'MODE(clicks.MONTH(click_time))',
    'MODE(clicks.WEEKDAY(click_time))'
])

In [17]:
for (train_file, feature_file) in zip(train_files, feature_files):
    df_train = pd.read_csv(train_file, usecols=to_read, dtype=dtypes, parse_dates=to_parse)
    df_feature = pd.read_csv(feature_file)
    df_feature = df_feature.set_index('channel')
    df_train = df_train.join(df_feature, on='channel')
#     to_drop = [
#         'click_time'
#     ]
#     for c in df_train.columns:
#         if c.startswith(to_drop_sw):
#             to_drop.append(c)
        
#     df_train.drop(columns=to_drop, inplace=True)
    to_select = ['is_attributed', 'os', 'device', 'channel', 'app']
    for c in df_train.columns:
        if c.startswith(to_select_sw):
            to_select.append(c)
    df_train = df_train[to_select]
    
    name, ext = os.path.splitext(train_file)
    output_file = f"{name}_features{ext}"
    print(f"Writing {output_file}")
    df_train.to_csv(output_file, index=False)
    del df_feature
    del df_train
    gc.collect()

Writing ../data/interim/train_2017-11-08_0000_features.csv
Writing ../data/interim/train_2017-11-08_0100_features.csv
Writing ../data/interim/train_2017-11-08_0200_features.csv
Writing ../data/interim/train_2017-11-08_0300_features.csv
Writing ../data/interim/train_2017-11-08_0400_features.csv
